This notebook was inpired by [this LlamaIndex notebook](https://colab.research.google.com/drive/1R41zIhVybCNqg67eVEPuyLeMp_HYwTlA?usp=sharing)

Making some changes to it with the only intention of trying ideas and learning.

Notice that I am assuming you have the relevant API_KEYs as environmental variables.

In [2]:
from bubls.utils.agents.general_function_tools import (
    multiply_tool,
    add_tool,
    subtract_tool
)
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

## Define tools

In [3]:

def useless(a: str) -> str:
    """Toy useless function."""
    pass

useless_tools = [
    FunctionTool.from_defaults(fn=useless, name=f"useless_{str(idx)}")
    for idx in range(100)
]

tools = [multiply_tool] + [add_tool] + [subtract_tool] + useless_tools

## Using Retrieval Augmented to reduce latency and cost

In [4]:
obj_index = ObjectIndex.from_objects(
    tools,
    index_cls=VectorStoreIndex,
)
llm_gpt = OpenAI(model="gpt-4")

In [17]:
%%time
gpt_worker_with_ra = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    llm=llm_gpt,
    verbose=True,
    allow_parallel_tool_calls=False
)
gpt_agent = AgentRunner(gpt_worker_with_ra)
response = gpt_agent.chat("What is 20+(2*4)? Calculate step by step.")
print("---\n", "Response\n", response)

Added user message to memory: What is 20+(2*4)? Calculate step by step.


=== Calling Function ===
Calling function: multiply with args: {"a": 2, "b": 4}
=== Function Output ===
8
=== Calling Function ===
Calling function: add with args: {"a": 20, "b": 8}
=== Function Output ===
28
=== LLM Response ===
The result of 20+(2*4) is 28.
---
 Response
 assistant: The result of 20+(2*4) is 28.
CPU times: user 212 ms, sys: 8.64 ms, total: 221 ms
Wall time: 4.99 s


In [18]:
%%time
gpt_worker_without_ra = FunctionCallingAgentWorker.from_tools(
    all_tools,
    llm=llm_gpt,
    verbose=True,
    allow_parallel_tool_calls=False
)
gpt_agent_2 = AgentRunner(gpt_worker_without_ra)
response = gpt_agent_2.chat("What is 20+(2*4)? Calculate step by step.")
print("---\n", "Response\n", response)

Added user message to memory: What is 20+(2*4)? Calculate step by step.
=== Calling Function ===
Calling function: multiply with args: {"a": 2, "b": 4}
=== Function Output ===
8
=== Calling Function ===
Calling function: add with args: {"a": 20, "b": 8}
=== Function Output ===
28
=== LLM Response ===
The result of the calculation 20+(2*4) is 28.
---
 Response
 assistant: The result of the calculation 20+(2*4) is 28.
CPU times: user 189 ms, sys: 3.79 ms, total: 193 ms
Wall time: 6.51 s
